## **Apache Beam**

Installing Apache beam interactive

In [1]:
!pip install apache-beam[interactive]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.9 MB 5.1 MB/s 
     |████████████████████████████████| 2.4 MB 35.2 MB/s 
     |████████████████████████████████| 47 kB 4.9 MB/s 
     |████████████████████████████████| 270 kB 59.7 MB/s 
     |████████████████████████████████| 508 kB 63.1 MB/s 
     |████████████████████████████████| 151 kB 60.3 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 138 kB 72.1 MB/s 
     |████████████████████████████████| 186 kB 52.8 MB/s 
     |████████████████████████████████| 423 kB 34.5 MB/s 
     |████████████████████████████████| 793 kB 47.1 MB/s 
     |████████████████████████████████| 1.6 MB 53.4 MB/s 
     |████████████████████████████████| 1.0 MB 54.5 MB/s 
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=d764509550a6fe0e2017cb2f771192e91ecd7c71db16a02518a595b8c95d939c
  S

Installing apache beam dataframe

In [1]:
!pip install apache_beam[dataframe]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install --quiet apache-beam

Importing all the necessary libraries

In [3]:
import apache_beam as beam
import pandas as pd
from apache_beam.dataframe.io import read_csv
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner

Read the csv data and filter the customers that purchased chocolate and put it into an output file

In [23]:
p1 = beam.Pipeline()
visit_count = (
  
  p1
  |beam.dataframe.io.read_csv('/content/grocery.csv')
  #|beam.io.Read('/content/grocery.txt')

  |beam.Map(lambda record: record.split(','))
  |beam.Filter(lambda record: record[3] == 'chocolate')
  |beam.Map(lambda record: (record[1], 1))
  |beam.CombinePerKey(sum)
  
  |beam.io.WriteToText('out_data.txt')
)


In [24]:
p1.run()

calculating the customers who brought both choclotates and yogurt and pushing into 

In [34]:
import apache_beam as beam


p2 = beam.Pipeline()


input_collection = (

    p2
    | "Read input data" >> beam.io.ReadFromText('/content/grocery.txt')
    | "Split rows into columns" >> beam.Map(lambda record: record.split(','))
                  )

chocolate_buyers_count = (
    input_collection
    | 'Filter Chocolate buyers' >> beam.Filter(lambda record: record[3] == 'chocolate')
    | 'Pair each chocolate buyer with 1' >> beam.Map(lambda record: ("chocolate, " +record[1], 1))
    | 'Aggregate all chocolate buyers' >> beam.CombinePerKey(sum)
    )

yogurt_buyers_count = (
    input_collection
    | 'Filter yogurt buyers' >> beam.Filter(lambda record: record[3] == 'yogurt')
    | 'Pair each buyer with 1' >> beam.Map(lambda record: ("yogurt, " +record[1], 1))
    | 'Aggregate all yogurt buyers' >> beam.CombinePerKey(sum)
    )

output =(
        (chocolate_buyers_count,yogurt_buyers_count)
  | beam.Flatten()
  | beam.io.WriteToText('data/both')
)


# **Composite transforms**

group all the purchases with chocolates and yogurt in the grocery dataset

In [32]:
import apache_beam as beam

class CustomTransform(beam.PTransform):
  
  def expand(self, input_coll):
    
    a = ( 
        input_coll
                       | 'Group and sum' >> beam.CombinePerKey(sum)
                       | 'Grocery sales' >> beam.Map(format_output)
              
    )
    return a

def SplitRow(element):
    return element.split(',')
  
def format_output(element):
  name, count = element
  return ', '.join((name,str(count)))

p4 = beam.Pipeline()

input_collection = ( 
                      p4
                      | "Read from text file" >> beam.dataframe.io.read_csv('/content/grocery.csv')
                      | "Split rows" >> beam.Map(SplitRow)
                   )

chocolate_count = (
                      input_collection
                      | 'Filter chocolate buyers' >> beam.Filter(lambda record: record[2] == 'chocolate')
                      | 'Pair each chocolate buyer with 1' >> beam.Map(lambda record: ("chocolate, " +record[1], 1))
                      | 'composite chocolate buyers' >> CustomTransform()
                      | 'Write results for chocolate' >> beam.io.WriteToText('chocolate_output.txt')
                 )

yogurt_count = (
                input_collection
                | 'Filter yogurt buyers' >> beam.Filter(lambda record: record[2] == 'Apples')
                | 'Pair each yogurt buyer with 1' >> beam.Map(lambda record: ("Apples, " +record[1], 1))
                | 'composite yogurt buyers' >> CustomTransform()
                | 'Write results for yogurt' >> beam.io.WriteToText('yogurt_output.txt')
           )


p4.run()